## Packages and Assets

In [1]:
from gensim.models import Word2Vec
import json
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
import seaborn as sns
import keras_tuner as kt
from keras.optimizers import SGD

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

import matplotlib.pyplot as plt

from keras.preprocessing.text import tokenizer_from_json
import os, sys

In [2]:
print(sys.executable)
print(os.getcwd())

C:\Users\Guilherme\.conda\envs\nlp2\python.exe
C:\Users\Guilherme\Documents\Programming\Python\DataScience\TCC\pipeline\supervised_deep_models


## Dependencies

In [3]:
with open('../../assets/deep_assets/tokenizer.json', 'r', encoding='utf-8') as f:
    tokenizer_json = f.read()
    tokenizer = tokenizer_from_json(tokenizer_json)
    word_index = tokenizer.word_index



In [4]:
model_we = Word2Vec.load('../../assets/deep_assets/word2vec.model')

print(model_we.wv.most_similar('itau'))

# List of nparrays of size 300
embeddings_dict = {}
for word in model_we.wv.index_to_key:
    embeddings_dict[word] = model_we.wv[word]

embeddings_on_this_context = np.zeros((len(word_index), 300))
for word, i in word_index.items():
    embeddings_vector = embeddings_dict.get(word)
    if embeddings_vector is not None:
        embeddings_on_this_context[i - 1] = embeddings_vector


[('itub4', 0.9894438982009888), ('banco', 0.9625048041343689), ('bbas3', 0.9558351635932922), ('bbdc4', 0.9518388509750366), ('bradesco', 0.9462378621101379), ('santander', 0.9438444972038269), ('unibanco', 0.9431132078170776), ('valioso', 0.9285449981689453), ('xpbr31', 0.9095613956451416), ('pagara', 0.8953273892402649)]


In [5]:
train = pd.read_csv('../../assets/data/splits/train/padded.csv')

## Functions

In [6]:
def index2word(word_index):
    index_word = {}
    for key in word_index:
        index_word[word_index[key]] = key
    return index_word


def seq2text(seq, index_word):
    text = []
    for index in seq:
        text.append(index_word[index])
    return text


def show_confusion_matrix(cm):
    print("Confusion Matrix")
    plt.figure(figsize=(10, 7))

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title('Confusion Matrix')
    plt.show()

## RNN Model

### Data transformation for model

In [7]:
X_train = train.to_numpy()[:, :-1]
y_train = train.to_numpy()[:, -1]

### Without Tuning and Early stopping

In [8]:
# model = Sequential([
#     Embedding(input_dim=len(word_index), output_dim= 300, input_length=X_train.shape[1], trainable=False, weights=[embeddings_on_this_context]),
#     Bidirectional(LSTM(64, return_sequences=True)),
#     # Dropout(0.4),
#     # Bidirectional(LSTM(hp.Choice('units',[32,64]))),
#     Bidirectional(LSTM(64)),
#     Dense(32, activation='relu'),
#     # Dropout(0.6),
#     Dense(4, activation='softmax')
# ])
#
# model.summary()

# from keras.optimizers import SGD
#
# loss = "sparse_categorical_crossentropy"
# optimizer = SGD(learning_rate=0.01)
# metrics = ['accuracy']
#
# model.compile(loss=loss,optimizer=optimizer,metrics=metrics)
# history = model.fit(X_train, y_train, epochs=25, validation_data=(X_val,y_val), verbose=2)

# history = model.fit(X_train, y_train, epochs=4000, validation_data=(X_val,y_val), verbose=2, callbacks=[es])


In [ ]:
# model = Sequential([
#     Embedding(input_dim=len(word_index), output_dim= 300, input_length=X_train.shape[1], trainable=False, weights=[embeddings_on_this_context]),
#     Bidirectional(LSTM(64, return_sequences=True)),
#     # Dropout(0.29),
#     # Bidirectional(LSTM(hp.Choice('units',[32,64]))),
#     Bidirectional(LSTM(64)),
#     Dense(32, activation='relu'),
#     # Dropout(0.73),
#     Dense(4, activation='softmax')
# ])
#
# model.summary()

# from keras.optimizers import SGD
#
# loss = "sparse_categorical_crossentropy"
# optimizer = SGD(learning_rate=0.01)
# metrics = ['accuracy']
#
# model.compile(loss=loss,optimizer=optimizer,metrics=metrics)
# history = model.fit(X_train, y_train, epochs=25, validation_data=(X_val,y_val), verbose=2)

# history = model.fit(X_train, y_train, epochs=4000, validation_data=(X_val,y_val), verbose=2, callbacks=[es])


## Tuning Architecture Hyperparams using RandomSearch and Early Stopping

### Test 1

In [19]:
def build_model(hp):
    model = Sequential([
        Embedding(input_dim=len(word_index), output_dim=300, input_length=X_train.shape[1], trainable=False,
                  weights=[embeddings_on_this_context]),
        Bidirectional(LSTM(16, return_sequences=True)),
        Dropout(hp.Float('rate_dp_1', 0.1, 0.9, step=0.1, default=0.5)),
        Bidirectional(LSTM(16)),
        Dense(64, 'tanh'),
        Dropout(hp.Float('rate_dp_1', 0.1, 0.9, step=0.1, default=0.5)),
        Dense(4, activation='softmax')
    ])
    loss = "sparse_categorical_crossentropy"
    # optimizer = SGD(learning_rate=0.01)
    metrics = ['accuracy']

    model.compile(loss=loss, optimizer='adam', metrics=metrics)
    return model


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

### Test 2

In [10]:
# def build_model(hp):
#     model = Sequential([
#     Embedding(input_dim=len(word_index), output_dim= 300, input_length=X_train.shape[1], trainable=False, weights=[embeddings_on_this_context]),
#     Bidirectional(LSTM(hp.Choice('units_bilstm_1',[16,32,64]), return_sequences=True)),
#     Dropout(hp.Float('rate_dp_1',0.5,0.9,step=0.1,default=0.5)),
#     Bidirectional(LSTM(hp.Choice('units_bilstm_2',[16,32,64]))),
#     Dense(hp.Choice('units_dense',[16,32,64]), hp.Choice('activation',['relu','sigmoid','tanh']) ),
#     Dropout(hp.Float('rate_dp_2',0.5,0.9,step=0.1,default=0.5)),
#     Dense(4, activation='softmax')
# ])
#     loss = "sparse_categorical_crossentropy"
#     # optimizer = SGD(learning_rate=0.01)
#     metrics = ['accuracy']
#
#     model.compile(loss=loss,optimizer=hp.Choice('optimizer',['adam','sgd','rmsprop']),metrics=metrics)
#     return model
#
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [20]:
# amanha testar dropout 0.0, 0.1, 0.2
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1,
    directory='../../assets/deep_assets',
    overwrite=True,
    project_name='lstm_tuning')

tuner.search_space_summary()

Search space summary
Default search space size: 1
rate_dp_1 (Float)
{'default': 0.5, 'conditions': [], 'min_value': 0.1, 'max_value': 0.9, 'step': 0.1, 'sampling': 'linear'}


In [21]:
tuner.search(X_train, y_train, epochs=4000, validation_split=0.1, callbacks=[es], batch_size=32, verbose=2)

Trial 4 Complete [00h 04m 15s]
val_accuracy: 0.8291178345680237

Best val_accuracy So Far: 0.8426896929740906
Total elapsed time: 00h 11m 00s
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner.results_summary()
 # 0.845774233341217

Results summary
Results in ../../assets/deep_assets\lstm_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
rate_dp_1: 0.2
Score: 0.8426896929740906

Trial 1 summary
Hyperparameters:
rate_dp_1: 0.1
Score: 0.8365206718444824

Trial 0 summary
Hyperparameters:
rate_dp_1: 0.4
Score: 0.8322023153305054

Trial 3 summary
Hyperparameters:
rate_dp_1: 0.9
Score: 0.8291178345680237


In [50]:
best_model = tuner.get_best_models()[0]

### Evaluation

## Tuning fit hyperparamters using GridSearch

In [51]:
def architecture_tuned_model():
    model = Sequential([
        Embedding(input_dim=len(word_index), output_dim=300, input_length=X_train.shape[1], trainable=False,
                  weights=[embeddings_on_this_context]),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dropout(0.29),
        Bidirectional(LSTM(16)),
        Dense(64, 'tanh'),
        Dropout(0.73),
        Dense(4, activation='softmax')
    ])
    loss = "sparse_categorical_crossentropy"
    optimizer = 'adam'
    metrics = ['accuracy']

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model


In [52]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=architecture_tuned_model, verbose=0)

C:\Users\Guilherme\AppData\Local\Temp\ipykernel_38200\2405901899.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=architecture_tuned_model, verbose=0)


In [53]:
from sklearn.model_selection import GridSearchCV

es = EarlyStopping(monitor='accuracy', mode='min', verbose=1, patience=5)

param_grid = {'batch_size': [1,2,4,8]}

gs = GridSearchCV(estimator=model,
                  param_grid=param_grid,
                  cv=5)
gs.fit(X_train, y_train, callbacks=[es])
print(f"Best results for {model.__class__.__name__}")
print("Best Score of train set: " + str(gs.best_score_))
print("Best estimator: " + str(gs.best_estimator_))
print("Best parameter set: " + str(gs.best_params_))


KeyboardInterrupt: 

In [ ]:
gs.best_estimator_.model.history.history
# 0.7766810655593872

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(gs.best_estimator_.model.history.history['accuracy'])
ax1.plot(gs.best_estimator_.model.history.history['loss'])
ax1.set_title('Model Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Validation'], loc='upper left')

ax2.plot(gs.best_estimator_.model.history.history['loss'])
ax2.plot(gs.best_estimator_.model.history.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Validation'], loc='upper left')
plt.show()

### Exporting model

In [ ]:
model.save('../../assets/deep_assets/lstm_model')